In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [3]:
# Download dataset


data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [4]:
!wget $data -O data-week-3.csv 

--2025-10-16 09:29:13--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘data-week-3.csv’

data-week-3.csv     100%[===================>]  78.98K   174KB/s    in 0.5s    

2025-10-16 09:29:14 (174 KB/s) - ‘data-week-3.csv’ saved [80876/80876]



In [5]:
df = pd.read_csv('data-week-3.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [10]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [12]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [19]:
df[categorical_columns] = df[categorical_columns].fillna('NA')

In [20]:
df[categorical_columns].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [42]:
df[numerical_columns].isnull().sum()

number_of_courses_viewed      0
annual_income               181
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [43]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.columns

numerical_columns = list(df.dtypes[df.dtypes != 'object'].index)
numerical_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [44]:
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [46]:
df.annual_income = pd.to_numeric(df.annual_income, errors='coerce')
df.annual_income

0       79450.0
1       46992.0
2       78796.0
3       83843.0
4       85012.0
         ...   
1457        NaN
1458    65259.0
1459    45688.0
1460    71016.0
1461    92855.0
Name: annual_income, Length: 1462, dtype: float64

In [47]:
df.annual_income = df.annual_income.fillna(0.0)
df.annual_income

0       79450.0
1       46992.0
2       78796.0
3       83843.0
4       85012.0
         ...   
1457        0.0
1458    65259.0
1459    45688.0
1460    71016.0
1461    92855.0
Name: annual_income, Length: 1462, dtype: float64

In [48]:
df[numerical_columns].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### Question 1

What is the most frequent observation (mode) for the column industry?

In [50]:
df.industry.mode()

0    retail
Name: industry, dtype: object

### Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

    interaction_count and lead_score
    number_of_courses_viewed and lead_score
    number_of_courses_viewed and interaction_count
    annual_income and interaction_count

Only consider the pairs above when answering this question.

In [51]:
corr = df[numerical_columns].corr()

In [52]:
corr

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [53]:
pairs = {
    'interaction_count and lead_score': corr.loc['interaction_count','lead_score'],
    'number_of_courses_viewed and lead_score': corr.loc['number_of_courses_viewed','lead_score'],
    'number_of_courses_viewed and interaction_count': corr.loc['number_of_courses_viewed','interaction_count'],
    'annual_income and interaction_count': corr.loc['annual_income','interaction_count'],
}

In [54]:
print("\nQ2 - Correlation pairs:")
for k,v in pairs.items():
    print(f"  {k}: {v:.4f}")
# pick the pair with highest (absolute) correlation among these options:
best_pair = max(pairs.items(), key=lambda kv: abs(kv[1]))
print("Q2 - Best pair (by abs correlation):", best_pair[0], "value:", best_pair[1])



Q2 - Correlation pairs:
  interaction_count and lead_score: 0.0099
  number_of_courses_viewed and lead_score: -0.0049
  number_of_courses_viewed and interaction_count: -0.0236
  annual_income and interaction_count: 0.0270
Q2 - Best pair (by abs correlation): annual_income and interaction_count value: 0.027036472404814396


### Split the data

    Split your data in train/val/test sets with 60%/20%/20% distribution.
    Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
    Make sure that the target value converted is not in your dataframe.


In [56]:
# Prepare X, y and splits for Q3-Q6 (ensure converted is removed from features)
df_model = df.copy()
y = df_model['converted'].astype(int).values
X = df_model.drop(columns=['converted'])

In [69]:
from sklearn.model_selection import train_test_split

# Split using sklearn train_test_split seed=42 (60/20/20)
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)
print("\nSplit sizes -> train:", df_train.shape[0], "val:", df_val.shape[0], "test:", df_test.shape[0])



Split sizes -> train: 876 val: 293 test: 293


### Question 3

    Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
    Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

In [85]:
df_train.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [87]:
# Q3: mutual information between converted and categorical variables (training set only)
cat_cols_train = [c for c in df_train.columns if df_train[c].dtype == 'object']
print("Categorical columns:", cat_cols_train)

Categorical columns: ['lead_source', 'industry', 'employment_status', 'location']


In [84]:
# convert categories to codes for mutual_info_classif
X_train_cat = X_train[cat_cols_train].apply(lambda s: s.astype('category').cat.codes)
mi = mutual_info_classif(X_train_cat, y_train, discrete_features=True, random_state=42)
mi_scores = dict(zip(cat_cols_train, mi))
mi_rounded = {k: round(v,2) for k,v in mi_scores.items()}

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [64]:
print("\nQ3 - Mutual information (rounded):")
for k,v in sorted(mi_rounded.items(), key=lambda kv: kv[1], reverse=True):
    print(f"  {k}: {v}")


Q3 - Mutual information (rounded):
  lead_source: 0.04
  industry: 0.01
  employment_status: 0.01
  location: 0.0


In [65]:
# pick among options:
candidates = ['industry','location','lead_source','employment_status']
print("Q3 - Candidate MIs:")
for c in candidates:
    print(f"  {c}: {mi_rounded.get(c, 0.0)}")
best_mi = max(candidates, key=lambda c: mi_rounded.get(c, 0.0))
print("Q3 - Best among given options:", best_mi, mi_rounded.get(best_mi))

Q3 - Candidate MIs:
  industry: 0.01
  location: 0.0
  lead_source: 0.04
  employment_status: 0.01
Q3 - Best among given options: lead_source 0.04


### Question 4

    Now let's train a logistic regression.
    Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
    Fit the model on the training dataset.
        To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
        model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

In [73]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [105]:
from sklearn.linear_model import LogisticRegression

In [106]:
# Q4: logistic regression with OHE and default params per instructions
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(df_train[cat_cols_train])

def prepare_ohe(dfX):
    num_cols_local = dfX.select_dtypes(include=[np.number]).columns.tolist()
    X_num = dfX[num_cols_local].to_numpy()
    if X_num.ndim == 1:
        X_num = X_num.reshape(-1, 1)
        
    X_cat = ohe.transform(dfX[cat_cols_train])
    if X_cat.ndim == 1:
        X_cat = X_cat.reshape(-1, 1)
    
    X_full = np.hstack([X_num, X_cat])
    feature_names = num_cols_local + list(ohe.get_feature_names_out(cat_cols_train))
    return X_full, feature_names
    
# def prepare_ohe(dfX):
#     num_cols_local = dfX.select_dtypes(include=[np.number]).columns.tolist()
#     X_num = dfX[num_cols_local].values
#     X_cat = ohe.transform(dfX[cat_cols_train])
#     X_full = np.hstack([X_num, X_cat])
#     feature_names = num_cols_local + list(ohe.get_feature_names_out(cat_cols_train))
#     return X_full, feature_names

X_train_ohe, feat_names = prepare_ohe(df_train)
X_val_ohe, _ = prepare_ohe(df_val)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_ohe, y_train)
y_val_pred = model.predict(X_val_ohe)
acc_val = accuracy_score(y_val, y_val_pred)
print("\nQ4 - Validation accuracy (unrounded):", acc_val, "rounded:", round(acc_val,2))


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

### Question 5

    Let's find the least useful feature using the feature elimination technique.
    Train a model using the same features and parameters as in Q4 (without rounding).
    Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
    For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

    'industry'
    'employment_status'
    'lead_score'

    Note: The difference doesn't have to be positive.


In [100]:
# Q5: feature elimination: baseline acc minus acc when excluding each original feature
baseline_acc = acc_val
feature_diffs = {}
for feat in X_train.columns:
    # create reduced feature matrix by removing this original feature
    if feat in X_train.select_dtypes(include=[np.number]).columns:
        # numeric: remove the exact column name from feat_names
        keep = [i for i, name in enumerate(feat_names) if name != feat]
    else:
        # categorical: remove all OHE columns that start with feat + '_'
        keep = [i for i, name in enumerate(feat_names) if not name.startswith(feat + '_')]
    X_tr_red = X_train_ohe[:, keep]
    X_val_red = X_val_ohe[:, keep]
    m = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    m.fit(X_tr_red, y_train)
    acc_red = accuracy_score(y_val, m.predict(X_val_red))
    feature_diffs[feat] = baseline_acc - acc_red

NameError: name 'acc_val' is not defined

In [101]:
print("\nQ5 - Differences (baseline - without feature):")
for k,v in sorted(feature_diffs.items(), key=lambda kv: abs(kv[1])):
    print(f"  {k}: {v:.6f}")

candidates_q5 = ['industry','employment_status','lead_score']
print("\nQ5 - Candidate diffs:")
for c in candidates_q5:
    print(f"  {c}: {feature_diffs.get(c)}")
best_q5 = min(candidates_q5, key=lambda c: abs(feature_diffs.get(c)))
print("Q5 - Feature with smallest difference among candidates:", best_q5, feature_diffs.get(best_q5))




Q5 - Differences (baseline - without feature):


NameError: name 'feature_diffs' is not defined

### Question 6

    Now let's train a regularized logistic regression.
    Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
    Train models using all the features as in Q4.
    Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

In [102]:

# Q6: test C values
C_values = [0.01,0.1,1,10,100]
C_results = {}
for C in C_values:
    m = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    m.fit(X_train_ohe, y_train)
    acc = accuracy_score(y_val, m.predict(X_val_ohe))
    C_results[C] = round(acc,3)


NameError: name 'LogisticRegression' is not defined

In [103]:
print("\nQ6 - Validation accuracies by C (rounded to 3 decimals):")
for c,v in C_results.items():
    print(f"  C={c}: {v}")
# pick best (if ties, smallest C)
best_acc = max(C_results.values())
best_Cs = sorted([c for c,a in C_results.items() if a==best_acc])
best_C = best_Cs[0]
print("Q6 - Best C:", best_C, "with val acc:", best_acc)


Q6 - Validation accuracies by C (rounded to 3 decimals):


ValueError: max() iterable argument is empty

In [104]:
# Also print test accuracy for best model optionally
best_model = LogisticRegression(solver='liblinear', C=best_C, max_iter=1000, random_state=42)
best_model.fit(X_train_ohe, y_train)
X_test_ohe, _ = prepare_ohe(X_test)
print("Test accuracy of best C model:", accuracy_score(y_test, best_model.predict(X_test_ohe)))



NameError: name 'LogisticRegression' is not defined

In [109]:
# Save as lead_homework.py and run with: python lead_homework.py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

# 1) Load data
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
df = pd.read_csv(url)
print("Rows, cols:", df.shape)

# 2) Data prep: categorical NaN -> 'NA', numeric NaN -> 0.0
df_prep = df.copy()
cat_cols = df_prep.select_dtypes(include=['object']).columns.tolist()
num_cols = df_prep.select_dtypes(include=[np.number]).columns.tolist()

for c in cat_cols:
    df_prep[c] = df_prep[c].fillna('NA')
for c in num_cols:
    df_prep[c] = df_prep[c].fillna(0.0)

# Q1: mode for industry
mode_industry = df_prep['industry'].mode().iloc[0]
print("Q1 - Mode of industry:", mode_industry)

# Q2: correlation among numeric features -> check specified pairs
corr = df_prep[num_cols].corr()
pairs = {
    'interaction_count and lead_score': corr.loc['interaction_count','lead_score'],
    'number_of_courses_viewed and lead_score': corr.loc['number_of_courses_viewed','lead_score'],
    'number_of_courses_viewed and interaction_count': corr.loc['number_of_courses_viewed','interaction_count'],
    'annual_income and interaction_count': corr.loc['annual_income','interaction_count'],
}
print("\nQ2 - Correlation pairs:")
for k,v in pairs.items():
    print(f"  {k}: {v:.4f}")
# pick the pair with highest (absolute) correlation among these options:
best_pair = max(pairs.items(), key=lambda kv: abs(kv[1]))
print("Q2 - Best pair (by abs correlation):", best_pair[0], "value:", best_pair[1])

# Prepare X, y and splits for Q3-Q6 (ensure converted is removed from features)
df_model = df_prep.copy()
y = df_model['converted'].astype(int).values
X = df_model.drop(columns=['converted'])

# Split using sklearn train_test_split seed=42 (60/20/20)
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.25, random_state=42)
print("\nSplit sizes -> train:", X_train.shape[0], "val:", X_val.shape[0], "test:", X_test.shape[0])

# Q3: mutual information between converted and categorical variables (training set only)
cat_cols_train = X_train.select_dtypes(include=['object']).columns.tolist()
print("\nCategorical columns:", cat_cols_train)
# convert categories to codes for mutual_info_classif
X_train_cat = X_train[cat_cols_train].apply(lambda s: s.astype('category').cat.codes)
mi = mutual_info_classif(X_train_cat, y_train, discrete_features=True, random_state=42)
mi_scores = dict(zip(cat_cols_train, mi))
mi_rounded = {k: round(v,2) for k,v in mi_scores.items()}
print("\nQ3 - Mutual information (rounded):")
for k,v in sorted(mi_rounded.items(), key=lambda kv: kv[1], reverse=True):
    print(f"  {k}: {v}")
# pick among options:
candidates = ['industry','location','lead_source','employment_status']
print("Q3 - Candidate MIs:")
for c in candidates:
    print(f"  {c}: {mi_rounded.get(c, 0.0)}")
best_mi = max(candidates, key=lambda c: mi_rounded.get(c, 0.0))
print("Q3 - Best among given options:", best_mi, mi_rounded.get(best_mi))

# Q4: logistic regression with OHE and default params per instructions
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(X_train[cat_cols_train])

def prepare_ohe(dfX):
    num_cols_local = dfX.select_dtypes(include=[np.number]).columns.tolist()
    
    X_num = dfX[num_cols_local].to_numpy()
    if X_num.ndim == 1:
        X_num = X_num.reshape(-1, 1)

    # Handle categorical features
    X_cat = ohe.transform(dfX[cat_cols_train])
    if isinstance(X_cat, np.ndarray) and X_cat.ndim == 1:
        X_cat = X_cat.reshape(-1, 1)
    elif not isinstance(X_cat, np.ndarray):
        X_cat = X_cat.toarray()  # For sparse output from OneHotEncoder

    # Combine numeric + categorical
    X_full = np.hstack([X_num, X_cat])
    feature_names = num_cols_local + list(ohe.get_feature_names_out(cat_cols_train))
    return X_full, feature_names

X_train_ohe, feat_names = prepare_ohe(X_train)
X_val_ohe, _ = prepare_ohe(X_val)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_ohe, y_train)
y_val_pred = model.predict(X_val_ohe)
acc_val = accuracy_score(y_val, y_val_pred)
print("\nQ4 - Validation accuracy (unrounded):", acc_val, "rounded:", round(acc_val,2))

# Q5: feature elimination: baseline acc minus acc when excluding each original feature
baseline_acc = acc_val
feature_diffs = {}
for feat in X_train.columns:
    # create reduced feature matrix by removing this original feature
    if feat in X_train.select_dtypes(include=[np.number]).columns:
        # numeric: remove the exact column name from feat_names
        keep = [i for i, name in enumerate(feat_names) if name != feat]
    else:
        # categorical: remove all OHE columns that start with feat + '_'
        keep = [i for i, name in enumerate(feat_names) if not name.startswith(feat + '_')]
    X_tr_red = X_train_ohe[:, keep]
    X_val_red = X_val_ohe[:, keep]
    m = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    m.fit(X_tr_red, y_train)
    acc_red = accuracy_score(y_val, m.predict(X_val_red))
    feature_diffs[feat] = baseline_acc - acc_red

print("\nQ5 - Differences (baseline - without feature):")
for k,v in sorted(feature_diffs.items(), key=lambda kv: abs(kv[1])):
    print(f"  {k}: {v:.6f}")

candidates_q5 = ['industry','employment_status','lead_score']
print("\nQ5 - Candidate diffs:")
for c in candidates_q5:
    print(f"  {c}: {feature_diffs.get(c)}")
best_q5 = min(candidates_q5, key=lambda c: abs(feature_diffs.get(c)))
print("Q5 - Feature with smallest difference among candidates:", best_q5, feature_diffs.get(best_q5))

# Q6: test C values
C_values = [0.01,0.1,1,10,100]
C_results = {}
for C in C_values:
    m = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    m.fit(X_train_ohe, y_train)
    acc = accuracy_score(y_val, m.predict(X_val_ohe))
    C_results[C] = round(acc,3)

print("\nQ6 - Validation accuracies by C (rounded to 3 decimals):")
for c,v in C_results.items():
    print(f"  C={c}: {v}")
# pick best (if ties, smallest C)
best_acc = max(C_results.values())
best_Cs = sorted([c for c,a in C_results.items() if a==best_acc])
best_C = best_Cs[0]
print("Q6 - Best C:", best_C, "with val acc:", best_acc)

# Also print test accuracy for best model optionally
best_model = LogisticRegression(solver='liblinear', C=best_C, max_iter=1000, random_state=42)
best_model.fit(X_train_ohe, y_train)
X_test_ohe, _ = prepare_ohe(X_test)
print("Test accuracy of best C model:", accuracy_score(y_test, best_model.predict(X_test_ohe)))


Rows, cols: (1462, 9)
Q1 - Mode of industry: retail

Q2 - Correlation pairs:
  interaction_count and lead_score: 0.0099
  number_of_courses_viewed and lead_score: -0.0049
  number_of_courses_viewed and interaction_count: -0.0236
  annual_income and interaction_count: 0.0270
Q2 - Best pair (by abs correlation): annual_income and interaction_count value: 0.027036472404814396

Split sizes -> train: 876 val: 293 test: 293

Categorical columns: ['lead_source', 'industry', 'employment_status', 'location']

Q3 - Mutual information (rounded):
  lead_source: 0.04
  industry: 0.01
  employment_status: 0.01
  location: 0.0
Q3 - Candidate MIs:
  industry: 0.01
  location: 0.0
  lead_source: 0.04
  employment_status: 0.01
Q3 - Best among given options: lead_source 0.04

Q4 - Validation accuracy (unrounded): 0.6996587030716723 rounded: 0.7

Q5 - Differences (baseline - without feature):
  industry: 0.000000
  employment_status: 0.003413
  lead_source: -0.003413
  lead_score: -0.006826
  location: -0